# 1. IMDB (순환신경망) - GRU

In [1]:
from tensorflow import keras
from keras.datasets import imdb

In [2]:
# 가장 자주 나타나는 단어 300개만 사용
(train_data, train_target), (test_data, test_target) = imdb.load_data(num_words=300)

17473536/17464789 [==============================] - 0s 0us/step


In [3]:
print(train_data.shape, train_target.shape)
print(test_data.shape, test_target.shape)

(25000,) (25000,)
(25000,) (25000,)


In [4]:
from sklearn.model_selection import train_test_split

train_data, val_data, train_target, val_target = train_test_split(train_data, train_target, test_size=0.2, random_state=0)

In [5]:
# 리뷰 길이를 100으로 맞추기
# 100보다 큰 경우 앞부분을 자르고, 100보다 작은 경우 앞부분에 0을 넣음
from keras.preprocessing.sequence import pad_sequences

train_seq = pad_sequences(train_data, maxlen=100)
val_seq = pad_sequences(val_data, maxlen=100)
print(train_seq.shape, val_seq.shape)   # (샘플 갯수, 토큰 개수(타임스텝 개수))

(20000, 100) (5000, 100)


In [6]:
from keras import models, layers

model = models.Sequential()
model.add(layers.Embedding(300, 5, input_length=100))  # (어휘 사전 크기, 임베딩 벡터 크기, 시퀀스 길이)
model.add(layers.GRU(8, return_sequences=True))  # 활성화함수(기본값) : tanh
model.add(layers.GRU(8))
model.add(layers.Dense(1, activation='sigmoid'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 5)            1500      
                                                                 
 gru (GRU)                   (None, 100, 8)            360       
                                                                 
 gru_1 (GRU)                 (None, 8)                 432       
                                                                 
 dense (Dense)               (None, 1)                 9         
                                                                 
Total params: 2,301
Trainable params: 2,301
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-gru.h5', save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)

history = model.fit(train_seq, train_target, epochs=100,
                    validation_data=(val_seq, val_target),
                    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/100
625/625 [==============================] - 15s 12ms/step - loss: 0.6051 - accuracy: 0.6528 - val_loss: 0.5248 - val_accuracy: 0.7414
Epoch 2/100
625/625 [==============================] - 7s 10ms/step - loss: 0.5076 - accuracy: 0.7556 - val_loss: 0.5706 - val_accuracy: 0.7124
Epoch 3/100
625/625 [==============================] - 7s 11ms/step - loss: 0.4961 - accuracy: 0.7635 - val_loss: 0.5020 - val_accuracy: 0.7518
Epoch 4/100
625/625 [==============================] - 7s 11ms/step - loss: 0.4911 - accuracy: 0.7642 - val_loss: 0.5136 - val_accuracy: 0.7422
Epoch 5/100
625/625 [==============================] - 7s 12ms/step - loss: 0.4878 - accuracy: 0.7646 - val_loss: 0.5014 - val_accuracy: 0.7512
Epoch 6/100
625/625 [==============================] - 7s 11ms/step - loss: 0.4842 - accuracy: 0.7690 - val_loss: 0.5066 - val_accuracy: 0.7478
Epoch 7/100
625/625 [==============================] - 7s 11ms/step - loss: 0.4820 - accuracy: 0.7714 - val_loss: 0.4978 - val_accuracy

In [9]:
test_seq = pad_sequences(test_data, maxlen=100)

In [10]:
model.evaluate(test_seq[:100], test_target[:100])

4/4 [==============================] - 0s 7ms/step - loss: 0.4597 - accuracy: 0.8000


[0.45967161655426025, 0.800000011920929]